In [ ]:
# Les bibliothèques nécessaires
import json
from collections import defaultdict # Permet d'initialiser des valeurs par défaut pour les nouvelles clés

!pip install gradio # Important à installer car ce n'est pas déjà prédéfinie
import gradio as gr # Pour l'interface graphique Web

import plotly.graph_objects as go # Pour faire le plot du Map
import re

import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.8 MB/s eta 0:00:00


# **Première partie** : on importe le JSON et on fait un bref appercue

---



In [ ]:
#On importe le fichier
with open('/content/CleanAndStrutured.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# On affiche les clés et leurs valeurs
if isinstance(data, dict):
    print("Aperçu des clés et valeurs :")
    for key, value in data.items():
        print(f"{key}: {value}")
elif isinstance(data, list):
    print("Aperçu des objets dans la liste :")
    for idx, item in enumerate(data[:5]):  # Montre les 5 premiers objets
        print(f"Objet {idx + 1}:")
        for key, value in item.items():
            print(f"  {key}: {value}")
else:
    print("Type de données non supporté.")

Aperçu des objets dans la liste :
Objet 1:
  url: https://www.reussir.fr/fruits-legumes/un-concept-innovant-pour-ecouler-mes-produits
  description: Un concept innovant pour écouler des produits locaux et vintage à Strasbourg.
  localisation: Strasbourg, France
  contacts: non renseigné
  type_de_produits: Produits locaux, fruits, légumes, objets vintage
  categorie: Commerce local
  update: 2023-01-15
Objet 2:
  url: https://www.dna.fr/politique/2025/01/15/un-vrai-elan-de-solidarite-pour-l-epicerie-sociale
  description: Un élan de solidarité pour l'épicerie sociale intercommunale de Gundershoffen.
  localisation: Rue Principale, Gundershoffen, France
  contacts: non renseigné
  type_de_produits: Produits alimentaires à tarif réduit
  categorie: Epicerie solidaire
  update: 2025-01-15
Objet 3:
  url: https://www.dna.fr/societe/2021/12/03/l-epicerie-sociale-ne-connait-pas-la-crise
  description: L'épicerie sociale de Gundershoffen continue d'assurer son service pour les personnes préca

# **Deuxième partie**: conversation par mots-clés prédéfinie dans un Fichier JSON

In [ ]:
data_path = '/content/CleanAndStrutured.json'
keywords_path = '/content/keywords_config.json'

class SimpleQASystem: # On definit une class nommé "SimpleQASystem" le nom fait enfait réference à 'Simple Question Answer Stystèm', conçue pour être un système simple de questions-réponses basé sur des mots-clés.
    def __init__(self, data_path, keywords_path):
        # Chargement des données
        with open(data_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)

        # Chargement de la configuration des mots-clés
        with open(keywords_path, 'r', encoding='utf-8') as f:
            self.keywords = json.load(f)

        # Construction de l'index pour une recherche rapide
        self.index = self._build_index()

    def _build_index(self):
        """
        Construit un index inverse pour accélérer les recherches.
        """
        index = defaultdict(list)

        for idx, entry in enumerate(self.data):
            # Combine plusieurs champs textuels pour l'indexation
            text_fields = [
                entry.get('description', ''),
                entry.get('type_de_produits', ''),
                entry.get('categorie', '')
            ]
            search_text = ' '.join(text_fields).lower()

            # Indexation par localisation
            loc = entry.get('localisation', '').lower()
            if loc:
                index[loc].append(idx)

            # Indexation par catégories et sous-catégories
            for category in self.keywords['categories']:
                # Mots-clés principaux
                for kw in self.keywords['categories'][category]['keywords']:
                    if kw in search_text:
                        index[kw].append(idx)

                # Sous-catégories
                for subcat in self.keywords['categories'][category]['subcategories']:
                    for sub_kw in self.keywords['categories'][category]['subcategories'][subcat]:
                        if sub_kw in search_text:
                            index[sub_kw].append(idx)

        return index

    def _extract_address(self, entry):
        """
        Formate une adresse complète à partir des champs disponibles.
        """
        address_components = [
            entry.get('adresse', '').strip(),
            entry.get('code_postal', '').strip(),
            entry.get('ville', '').strip()
        ]

        # Filtre les composants vides et les combine
        full_address = ', '.join(filter(None, address_components))

        return full_address if full_address else "Adresse non renseignée"

    def _calculate_relevance(self, entry, question_keywords):
        """
        Calcule un score de pertinence pour une entrée.

        Args:
            entry (dict): Entrée de données
            question_keywords (set): Mots-clés de la question

        Returns:
            tuple: Score de pertinence (pour le tri)
        """
        entry_text = f"{entry['description']} {entry['type_de_produits']}".lower()

        # Nombre de mots-clés correspondants
        keyword_matches = sum(
            1 for kw in question_keywords
            if kw in entry_text
        )

        # Longueur du texte (priorité aux descriptions plus complètes)
        text_length = len(entry['description'])

        return (-keyword_matches, -text_length)  # Tri décroissant

    def find_best_match(self, question):
        """
        Trouve les meilleures réponses pour une question.

        Args:
            question (str): Question posée par l'utilisateur

        Returns:
            list: Liste des 3 meilleures réponses formatées
        """
        question_lower = question.lower()
        matched_indices = set()
        question_keywords = set()

        # 1. Détection des mots-clés de localisation
        loc_matches = []
        for loc_kw in self.keywords['localisation']['strasbourg']:
            if loc_kw in question_lower:
                loc_matches.extend(self.index.get(loc_kw, []))
                question_keywords.add(loc_kw)

        # 2. Détection des mots-clés thématiques
        for category in self.keywords['categories']:
            # Mots-clés principaux
            for kw in self.keywords['categories'][category]['keywords']:
                if kw in question_lower:
                    matched_indices.update(self.index.get(kw, []))
                    question_keywords.add(kw)

            # Sous-catégories
            for subcat in self.keywords['categories'][category]['subcategories']:
                for sub_kw in self.keywords['categories'][category]['subcategories'][subcat]:
                    if sub_kw in question_lower:
                        matched_indices.update(self.index.get(sub_kw, []))
                        question_keywords.add(sub_kw)

        # 3. Filtrage par localisation si spécifiée
        if loc_matches:
            matched_indices.intersection_update(loc_matches)

        # 4. Récupération et tri des résultats
        raw_results = [self.data[idx] for idx in matched_indices]

        # Tri par pertinence
        raw_results.sort(key=lambda x: self._calculate_relevance(x, question_keywords))

        # Formatage des résultats finaux
        formatted_results = []
        for entry in raw_results[:3]:  # On garde seulement les 3 meilleurs
            formatted_results.append({
                'titre': entry.get('description', '').split('.')[0][:50] + '...',  # Première phrase tronquée
                'description': entry.get('description', ''),
                'adresse_complete': self._extract_address(entry),
                'type': entry.get('type_de_produits', ''),
                'categorie': entry.get('categorie', ''),
                'url': entry.get('url', '')
            })

        return formatted_results

# Instanciation du système Qestion Answer
qa_system = SimpleQASystem(data_path, keywords_path)

def predict(question):
    results = qa_system.find_best_match(question)
    if results:
        output = ""
        for i, result in enumerate(results):
            output += f"**Réponse {i+1}:**\n"
            output += f"- **Titre:** {result['titre']}\n"
            output += f"- **Description:** {result['description']}\n"
            output += f"- **Adresse:** {result['adresse_complete']}\n"
            output += f"- **Type:** {result['type']}\n"
            output += f"- **Catégorie:** {result['categorie']}\n"
            if result['url']:
                output += f"- **URL:** {result['url']}\n"
            output += "\n"
        return output
    else:
        return "Aucun résultat pertinent trouvé."

frequent_questions = [
    "Où-est ce que je peux des paniers légumes à tarif étudiant à Strasbourg ?",
    "Où-est ce que je peux trouver des restaurants bio à Strasbourg ?",
    "Peux-tu me conseiller des épiceries solidaires à Strasbourg ?",
    "Où trouver des paniers bio à Strasbourg ?"
]


### **Sous-partie** : Amélioration du retour d'information

In [ ]:
def predict(question):
    results = qa_system.find_best_match(question)
    if results:
        output = ""
        for i, result in enumerate(results):
            output += f"**Réponse {i+1}:**\n"
            output += f"- **Titre:** {result['titre']}\n"
            output += f"- **Description:** {result['description']}\n"
            output += f"- **Adresse:** {result['adresse_complete']}\n"
            output += f"- **Type:** {result['type']}\n"
            output += f"- **Catégorie:** {result['categorie']}\n"
            if result['url']:
                output += f"- **URL:** {result['url']}\n"
            output += "\n"
            time.sleep(0.5)
            yield output  # Utilisation de yield pour l'affichage progressif
    else:
        yield "**Aucun résultat pertinent trouvé.**\n\nEssayez de reformuler votre question ou d'utiliser des mots-clés différents comme 'panier', 'local', 'bio', 'restaurant', 'épicerie', etc."


# **Troisième partie:**  Jeu du Budget Vert

In [ ]:
BUDGET_INITIAL = 50  # Définition du budget initial

ACTIVITES = {
    "Cinéma": 10,
    "Musique": 5,
    "Concerts": 20,
    "Festivals": 30,
    "Musées": 5,
    "Expositions": 5,
    "Manga": 5,
    "Animé": 5,
    "Jeux de société": 10,
    "Dessin": 5,
    "Jeux vidéo": 25,
    "Réseaux sociaux": 10,
    "YouTube": 10,
    "Sport": 15,
    "Cuisine": 10,
    "Vacances": 45,
    "École": 15,
    "Travail": 10,
}

ACTIVITES_DISPLAY = {
    "Cinéma": "🎬 Cinéma (10)",
    "Musique": "🎶 Musique (5)",
    "Concerts": "🎤 Concerts (20)",
    "Festivals": "🎉 Festivals (30)",
    "Musées": "🏛️ Musées (5)",
    "Expositions": "🖼️ Expositions (5)",
    "Manga": "📚 Manga (5)",
    "Animé": "📺 Animé (5)",
    "Jeux de société": "🎲 Jeux de société (10)",
    "Dessin": "✏️ Dessin (5)",
    "Jeux vidéo": "🎮 Jeux vidéo (25)",
    "Réseaux sociaux": "📱 Réseaux sociaux (10)",
    "YouTube": "▶️ YouTube (10)",
    "Sport": "⚽ Sport (15)",
    "Cuisine": "🍳 Cuisine (10)",
    "Vacances": "🏖️ Vacances (45)",
    "École": "🎒 École (15)",
    "Travail": "💼 Travail (10)",
}

def ajouter_ou_retirer_activite(activite_nom, budget_restant, activites_choisies_liste, activites_choisies_texte):
    cout = ACTIVITES.get(activite_nom, 0)
    if activite_nom in activites_choisies_liste:
        nouvelles_activites = [a for a in activites_choisies_liste if a != activite_nom]
        nouveau_budget = budget_restant + cout
    elif budget_restant >= cout:
        nouvelles_activites = activites_choisies_liste + [activite_nom]
        nouveau_budget = budget_restant - cout
    else:
        return budget_restant, activites_choisies_liste, activites_choisies_texte + f"\nBudget insuffisant pour {ACTIVITES_DISPLAY.get(activite_nom, activite_nom)} !"

    nouveau_texte = "\n- ".join([f"{ACTIVITES_DISPLAY.get(a, a)}" for a in nouvelles_activites])
    return nouveau_budget, nouvelles_activites, gr.Markdown(f"**Activités choisies :**\n- {nouveau_texte}")

def terminer_jeu(budget_restant, activites_choisies_liste):
    cout_total = BUDGET_INITIAL - budget_restant
    message = f"## Bilan de votre week-end éco-responsable :\n\n"
    message += f"Votre budget initial était de **{BUDGET_INITIAL}** points carbone.\n\n"
    if activites_choisies_liste:
        message += "Vous avez choisi les activités suivantes :\n- "
        message += "\n- ".join([f"{ACTIVITES_DISPLAY.get(a, a)}" for a in activites_choisies_liste]) + "\n\n"
        message += f"Le coût total de vos activités est de **{cout_total}** points carbone.\n\n"
    else:
        message += "Vous n'avez choisi aucune activité.\n\n"

    if budget_restant >= 0:
        message += f"Il vous reste **{budget_restant}** points carbone. Bravo pour votre week-end potentiellement sobre en carbone ! 🎉"
    else:
        message += f"Vous avez dépassé votre budget de **{-budget_restant}** points carbone. Essayez de faire des choix plus légers la prochaine fois ! 😥"
    return gr.Markdown(message, visible=True)

# **Quatrième Partie:** Creation d'un Map

### Data pour la locatisation

In [ ]:
data = [
    {
        "localisation": {
            "contacts": "non renseigné",
            "type_de_produits": "Produits locaux, fruits, légumes, objets vintage",
            "categorie": "Commerce local",
            "update": "2023-01-15"
        }
    },
    {
        "url": "https://www.dna.fr/politique/2025/01/15/un-vrai-elan-de-solidarite-pour-l-epicerie-sociale",
        "description": "Un élan de solidarité pour l'épicerie sociale intercommunale de Gundershoffen.",
        "localisation": {
            "adresse": "Rue Principale, Gundershoffen, France",
            "latitude": 48.8325,
            "longitude": 7.6667
        },
        "contacts": "non renseigné",
        "type_de_produits": "Produits alimentaires à tarif réduit",
        "categorie": "Epicerie solidaire",
        "update": "2025-01-15"
    },
    {
        "url": "https://www.francebleu.fr/emissions/l-eco-d-ici-ici-alsace/solibio-la-cooperative-grossiste-alsacienne-bio-a-le-vent-en-poupe-5368146",
        "description": "Solibio, une coopérative grossiste bio en Alsace, en pleine croissance.",
        "localisation": {
            "adresse": "Marché-Gare, Rue du Marché, Strasbourg, France",
            "latitude": 48.584614,
            "longitude": 7.734722
        },
        "contacts": "non renseigné",
        "type_de_produits": "Produits bio : fruits, légumes, produits laitiers, viande",
        "categorie": "Coopérative bio",
        "update": "2024-01-15"
    },
    {
        "url": "https://strasinfo.fr/2025/03/20/le-petit-marche-ouvre-ses-portes-a-cronenbourg-un-nouveau-concept-qui-mele-alimentaire-et-convivialite/",
        "description": "Le Petit Marché : un marché hybride mêlant produits locaux et animations à Cronenbourg.",
        "localisation": {
            "adresse": "Square Saint-Florent, Cronenbourg, Strasbourg, France",
            "latitude": 48.593889,
            "longitude": 7.709722
        },
        "contacts": "non renseigné",
        "type_de_produits": "Produits locaux : fruits, légumes, produits laitiers",
        "categorie": "Marché local",
        "update": "2025-03-20"
    },
    {
        "url": "https://www.vertici.fr",
        "description": "VERT ICI : salades, sandwichs et smoothies personnalisés avec des produits frais.",
        "localisation": {
            "adresse": "Centre commercial Aubette, Place Kléber, Strasbourg, France",
            "latitude": 48.5842,
            "longitude": 7.7469
        },
        "contacts": "non renseigné",
        "type_de_produits": "Salades, sandwichs, smoothies",
        "categorie": "Restauration rapide",
        "update": "non renseigné"
    }
]

In [ ]:

###### Localisation

def get_ecological_map_plotly():
    """Crée une carte Plotly des lieux écologiques."""
    locations = []
    names = []
    for item in data:
        if "localisation" in item and "latitude" in item["localisation"] and "longitude" in item["localisation"]:
            lat = item["localisation"]["latitude"]
            lon = item["localisation"]["longitude"]
            name = item.get("description", item["localisation"].get("adresse", "Lieu"))
            locations.append((lat, lon))
            names.append(name)

    if not locations:
        fig = go.Figure(go.Scattermapbox(lat=[48.584], lon=[7.744], mode='markers', text=["Strasbourg (par défaut)"]))
    else:
        lats, lons = zip(*locations)
        fig = go.Figure(go.Scattermapbox(lat=lats, lon=lons, mode='markers', text=names))

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_center={"lat": 48.584, "lon": 7.744},  # Centre sur Strasbourg par défaut
        mapbox_zoom=10,
        margin={"r": 0, "t": 0, "l": 0, "b": 0}
    )
    return fig


# **Cinquième partie** : l'interface web avec des onglets

In [ ]:
with gr.Blocks(theme=gr.themes.Soft(), title="IA Verte Locale : Strasbourg") as iface:
    # Onglet de la recherche
    with gr.Tab("Recherche"):
        gr.Markdown(
            """
            <div style="text-align: center;">
                <h1>IA Verte Locale : Strasbourg</h1>
            </div>
            <div style="display: flex; justify-content: center;">
                <img src="https://wehost.fr/wp-content/uploads/2024/04/Strasbourg-1024x683.png" alt="Vue de Strasbourg" style="max-width: 500px; width: 80%;">
            </div>

            Posez une question et trouvez les meilleures correspondances dans notre base de données.

            **Conseil :** Pour des résultats plus précis, essayez d'inclure des mots-clés comme 'panier', 'local', 'bio', 'restaurant', 'épicerie', etc.
            """
        )
        with gr.Row():
            question_input = gr.Textbox(label="Posez votre question :")
            output_results = gr.Markdown(label="Meilleures réponses :")

        question_input.change(fn=predict, inputs=question_input, outputs=output_results) # Appel direct à predict

        gr.Markdown("## Questions fréquemment posées")
        for question in frequent_questions:
            gr.Markdown(f"- {question}")
#Onglet du jeu
    with gr.Tab("Jeu du Budget Vert : Week-end Éco-Loisirs"):
        gr.Markdown(
            """
            ## Planifiez votre week-end éco-responsable à Strasbourg !

            Bienvenue dans le jeu du Budget Vert. L'objectif est de planifier un week-end d'activités tout en respectant un budget carbone de **50 points**.

            **Choisissez vos activités en cliquant dessus dans la liste ci-dessous.** Les activités sélectionnées apparaîtront dans la section 'Activités choisies', et votre budget sera mis à jour en temps réel.

            Essayez de choisir des activités qui vous plaisent tout en minimisant votre impact carbone ! Une fois que vous avez terminé, cliquez sur le bouton 'Terminer mon week-end' pour voir votre bilan.
            """
        )
        budget_restant = gr.State(BUDGET_INITIAL)
        activites_choisies_liste = gr.State([])
        activites_choisies_texte = gr.Markdown("**Activités choisies :**")
        budget_affichage = gr.Markdown(f"Budget carbone restant : **{BUDGET_INITIAL}** points")

        with gr.Column():
            gr.Markdown("### Liste des activités :")
            for activite_nom, cout in ACTIVITES.items():
                bouton = gr.Button(ACTIVITES_DISPLAY.get(activite_nom, f"{activite_nom} ({cout})"), variant="secondary")
                bouton.click(
                    fn=ajouter_ou_retirer_activite,
                    inputs=[gr.Textbox(value=activite_nom, visible=False), budget_restant, activites_choisies_liste, activites_choisies_texte],
                    outputs=[budget_restant, activites_choisies_liste, activites_choisies_texte]
                )

            budget_restant.change(fn=lambda budget: f"Budget carbone restant : **{budget}** points", inputs=budget_restant, outputs=budget_affichage)

            gr.Markdown("---")
            gr.Markdown("### Vos choix :")
            output_activites_choisies = activites_choisies_texte

            terminer_button = gr.Button("Terminer mon week-end", variant="primary")
            resultat_jeu = gr.Markdown(visible=False)
            terminer_button.click(
                fn=terminer_jeu,
                inputs=[budget_restant, activites_choisies_liste],
                outputs=resultat_jeu
            )

            resultat_jeu # Affichage du résultat en dernier dans la colonne
#Onglet pour la localisation
    with gr.Tab("Carte des Lieux Écologiques"):
        plotly_map = gr.Plot(get_ecological_map_plotly())

iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://328007eaf593145718.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
